Requires the name of the flight and maia file (including .csv for flight and .log for Maia
Script below takes as an input the flight logs from the first few flights, when mission start and finish are not noted in the flight log but the phrase "SUCCESSED SIMPLE_SHOT" is and that has a corresponding real time attached. Will use the speed of the UAV to determine when it has finished flying its mission, when the speed is no longer 2ms-1, and then use the altitude to inform this decision. Then makes a new flight file with only the relevant mission lines, then makes a new MAIA file with only the relevant mission lines and only the four columns Image Name / Long / Lat / Altitude

In [10]:
flightFileName = 'FLY216_20170718_ES_P2.csv'
maiaFileName = '20170718165614070_02-00000.log'
newFlightFileName = 'justMission' + flightFileName
newMaiaFileName = 'justMission' + maiaFileName[:-3] + 'csv'

with open(flightFileName) as flightFile:      
    flightFileLines = flightFile.read().split("\n")
    print("Number of lines is {}".format(len(flightFileLines)))
    
camShotLine = []
word = 'SUCCESSED SIMPLE_SHOT'
   
for i,line in enumerate(flightFileLines):
    if word in line: # or word in line.split() to search for full words
        camShotLine.append(i)
        print("Word \"{}\" found in line {}".format(word, i))
                     
potMatch = len(camShotLine)
potMatchLines = [flightFileLines[x] for x in camShotLine]

dateWord = '|#SiF%|2'

times = []

for y in range(potMatch):
    charDateWord = potMatchLines[y][(potMatchLines[y].find(dateWord)+7):((potMatchLines[y].find(dateWord)+15))]
    if potMatchLines[y].find(charDateWord+".0")>1:
        times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".0")+9):((potMatchLines[y].find(charDateWord+".0")+17))])
    elif potMatchLines[y].find(charDateWord+".1")>1:  
        times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".1")+9):((potMatchLines[y].find(charDateWord+".1")+17))])
    elif potMatchLines[y].find(charDateWord+".2")>1:
        times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".2")+9):((potMatchLines[y].find(charDateWord+".2")+17))])

print(times)
flightTimes = [(flightFileLines[x]).split(',')[1] for x in camShotLine]

#This is the new code START

missionSpeed = 2
missionSpeedBumpiness = 0.3
missionAlt = 60
missionAltBumpiness = 2.5
flightSpeeds = [(flightFileLines[x]).split(',')[27] for x in range(len(flightFileLines)-1)]
flightRelHeight = [(flightFileLines[x]).split(',')[10] for x in range(len(flightFileLines)-1)]

def speedChecker(vals, heightVals):
    cnt = 0
    for i in vals:
        if missionSpeed - missionSpeedBumpiness < i < missionSpeed + missionSpeedBumpiness:
            cnt = cnt + 1
    if cnt == len(vals):
        if altitudeChecker(heightVals) == 1:
            return 1
        else:
            return 0
    else: 
        return 0

def altitudeChecker(vals):  
    cnt = 0
    for i in vals:
        if missionAlt - missionAltBumpiness < i < missionAlt + missionAltBumpiness:
            cnt = cnt + 1
    if cnt == len(vals):
        return 1
    else: 
        return 0
    
check = 0
counter = 1
while check == 0:
    speedVals = [float(flightSpeeds[x]) for x in range(counter, counter+30)]
    heightVals = [float('0' + flightRelHeight[x]) for x in range(counter, counter+30)]
    counter = counter + 1
    check = speedChecker(speedVals, heightVals)
missStartLine = counter
print(missStartLine)

check = 0
counter = len(flightFileLines) - 1
while check == 0:
    speedVals = [float(flightSpeeds[x]) for x in range(counter-30, counter)]
    heightVals = [float('0' + flightRelHeight[x]) for x in range(counter-30, counter)]
    counter = counter - 1
    check = speedChecker(speedVals, heightVals)
missEndLine = counter
print(missEndLine)       

#This is the new code END need missStartLine and missEndLine
missStartFlightTime = (flightFileLines[missStartLine]).split(',')[1]
missFinishFlightTime = (flightFileLines[missEndLine]).split(',')[1]
flightTimeDiffs = [float(missStartFlightTime)-float(flightTimes[0]), float(missFinishFlightTime)-float(flightTimes[0])]
print('Flight time diffs are:', flightTimeDiffs)
realFlightTimes = [0,0]
for j in range(2):
    timeHolder = (times[0]).split(':')
    for i in range(3):
        timeHolder[i] = float(timeHolder[i])
    timeHolder[2] = timeHolder[2] + flightTimeDiffs[j]
    while timeHolder[2] >= 60:
        timeHolder[2] = timeHolder[2] - 60
        timeHolder[1] = timeHolder[1] + 1
        while timeHolder[1] >= 60:
            timeHolder[1] = timeHolder[1] - 60
            timeHolder[0] = timeHolder[0] + 1
    while timeHolder[2] <= 0:
        timeHolder[2] = timeHolder[2] + 60
        timeHolder[1] = timeHolder[1] - 1
        while timeHolder[1] <= 0:
            timeHolder[1] = timeHolder[1] + 60
            timeholder[0] = timeHolder[0] - 1
    realFlightTimes[j] = timeHolder
print('realFlight times are:', realFlightTimes)

with open(maiaFileName) as maiaFile:      
    maiaFileLines = maiaFile.read().split("\n")
    numLines = len(maiaFileLines)
    maiaFlightTimes = [(maiaFileLines[x]).split('\t')[2] for x in range(numLines-1)]

    
fStart = str(int(realFlightTimes[0][0])).zfill(2) + ':' + str(int(realFlightTimes[0][1])).zfill(2) + ':' \
        + str(int(realFlightTimes[0][2])).zfill(2)
fEnd = str(int(realFlightTimes[1][0])).zfill(2) + ':' + str(int(realFlightTimes[1][1])).zfill(2) + ':' \
        + str(int(realFlightTimes[1][2])).zfill(2)

for i in range(numLines-1):
    if maiaFlightTimes[i].startswith(fStart):
        maiaStartLine = i
        print('Mission starts on line:', i)
        
for j in range(maiaStartLine, numLines-1):
    if maiaFlightTimes[j].startswith(fEnd):
        maiaEndLine = j
        print('Mission ends on line:', j)
        
with open(newFlightFileName, 'w') as outputFlightFile:
    firstRow = flightFileLines[0] + "\n"
    outputFlightFile.write(firstRow)
    for i in range(missStartLine-20, missEndLine+20):
        row = flightFileLines[i] + "\n"
        outputFlightFile.write(row)
        
maiaImageNames = [(maiaFileLines[x]).split('\t')[3] for x in range(numLines-1)]
maiaLongs = [(maiaFileLines[x]).split('\t')[7] for x in range(numLines-1)]
maiaLats = [(maiaFileLines[x]).split('\t')[9] for x in range(numLines-1)]
maiaAlts = [(maiaFileLines[x]).split('\t')[11] for x in range(numLines-1)]

with open(newMaiaFileName, 'w') as outputMaiaFile:
    firstRow = maiaImageNames[0] + "," + maiaLongs[0] + "," + maiaLats[0] + "," + maiaAlts[0] + "\n"
    outputMaiaFile.write(firstRow)
    for i in range(maiaStartLine, maiaEndLine):
        row = maiaImageNames[i] + "," + '-' + maiaLongs[i] + "," + maiaLats[i] + "," + maiaAlts[i] + "\n"
        outputMaiaFile.write(row)
        
print('All done! :)')



Number of lines is 29363
Word "SUCCESSED SIMPLE_SHOT" found in line 4157
Word "SUCCESSED SIMPLE_SHOT" found in line 26079
['16:58:06', '17:10:30']
4244
25857
Flight time diffs are: [2.9549999999999983, 736.24]
realFlight times are: [[16.0, 58.0, 8.954999999999998], [17.0, 10.0, 22.24000000000001]]
Mission starts on line: 115
Mission ends on line: 849
All done! :)
